# UYG332 Image Processing Final Project

**Prepared by:** _Your Name_  
**Student ID:** _XXXXXXXX_

## Setup and Imports

Ensure all provided images are in the same directory as this notebook (e.g., `/mnt/data/`).

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

plt.rcParams['figure.figsize'] = (6,6)

def show(img, cmap=None, title=''):
    plt.figure()
    if cmap:
        plt.imshow(img, cmap=cmap)
    else:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()


## Problem 1: Colour Patch on tf2_engineer.jpg
**Tasks:**
1. Read & display color image.
2. Find centre & print intensity.
3. Place 30×40 patch (#329ea8) around centre.
4. Print intensity at centre of patch.
5. Display result.

In [ ]:
# 1. Read and display
img1 = cv2.imread('/mnt/data/tf2_engineer.jpg')
show(img1, title='Original tf2_engineer.jpg')

# 2. Centre & intensity
h, w = img1.shape[:2]
yc, xc = h//2, w//2
print(f"Centre: (y={yc}, x={xc}) -> Intensity (B,G,R):", img1[yc, xc])

# 3. Patch (#329ea8 -> BGR=(168,158,50))
patch_color = (168,158,50)
y1, y2 = yc-15, yc+15
x1, x2 = xc-20, xc+20
img1p = img1.copy()
cv2.rectangle(img1p, (x1,y1), (x2,y2), patch_color, thickness=-1)

# 4. Intensity at patch centre
print("Patch centre intensity (B,G,R):", img1p[yc, xc])

# 5. Display patched image
show(img1p, title='Patched Image')


## Problem 2: Negative of einstein.tiff
**Tasks:**
1. Read & display grayscale.
2. Compute negative.
3. Display negative.
4. Print 5 random pixel values before/after.

In [ ]:
# 1. Read & display
img2 = cv2.imread('/mnt/data/einstein.png', cv2.IMREAD_GRAYSCALE)
show(img2, cmap='gray', title='Einstein Original')

# 2. Negative
neg2 = 255 - img2

# 3. Display negative
show(neg2, cmap='gray', title='Einstein Negative')

# 4. Sample 5 pixels
coords = [(random.randint(0,img2.shape[0]-1),
           random.randint(0,img2.shape[1]-1)) for _ in range(5)]
print("Original -> Negative")
for y,x in coords:
    print(f"({y},{x}): {img2[y,x]} -> {neg2[y,x]}")


## Problem 3: Log & Inverse Log on pout.tiff
**Tasks:**
1. Read & display grayscale.
2. Log transform & display.
3. Inverse log on original & display.
4. Inverse log on log image & display.
5. Comment on results.

In [ ]:
# 1. Read & display
img3 = cv2.imread('/mnt/data/pout.png', cv2.IMREAD_GRAYSCALE)
show(img3, cmap='gray', title='pout.tiff Original')

# 2. Log transform
c = 255.0 / np.log(1 + img3.max())
log_img = (c * np.log1p(img3)).astype(np.uint8)
show(log_img, cmap='gray', title='Log Transform')

# 3. Inverse log of original
inv1 = (np.expm1(img3 / c)).clip(0,255).astype(np.uint8)
show(inv1, cmap='gray', title='Inv-Log of Original')

# 4. Inverse log of log image
inv2 = (np.expm1(log_img / c)).clip(0,255).astype(np.uint8)
show(inv2, cmap='gray', title='Inv-Log of Log Image')

# 5. Comments
print("Log enhances dark areas; inverse-log recovers original approximately.")


## Problem 4: Unsharp Masking on moon.tiff
**Tasks:**
1. Read & display grayscale.
2. Spatial unsharp for k=0.2,0.5,1.0.
3. Frequency unsharp same k values.
4. Display results side by side.

In [ ]:
# Read and display
img4 = cv2.imread('/mnt/data/moon.png', cv2.IMREAD_GRAYSCALE)
show(img4, cmap='gray', title='moon.tiff Original')

# Spatial unsharp
blur = cv2.GaussianBlur(img4, (9,9), 0)
mask = cv2.subtract(img4, blur)
for k in [0.2, 0.5, 1.0]:
    g_sp = cv2.addWeighted(img4, 1.0, mask, k, 0)
    show(g_sp, cmap='gray', title=f'Spatial Unsharp k={k}')

# Frequency unsharp
dft = cv2.dft(np.float32(img4), flags=cv2.DFT_COMPLEX_OUTPUT)
dft_shift = np.fft.fftshift(dft)
rows, cols = img4.shape
crow, ccol = rows//2, cols//2
D0 = 30
H = np.zeros((rows, cols), np.float32)
for u in range(rows):
    for v in range(cols):
        D = np.hypot(u-crow, v-ccol)
        H[u,v] = 1 - np.exp(-(D**2)/(2*D0**2))
H = cv2.merge([H,H])
for k in [0.2, 0.5, 1.0]:
    G = (1 + k*H) * dft_shift
    G_ishift = np.fft.ifftshift(G)
    img_back = cv2.idft(G_ishift)
    img_back = cv2.magnitude(img_back[:,:,0], img_back[:,:,1])
    img_back = cv2.normalize(img_back, None, 0,255, cv2.NORM_MINMAX).astype(np.uint8)
    show(img_back, cmap='gray', title=f'Freq Unsharp k={k}')


## Problem 5: Noise Removal on pcb.tiff
**Tasks:**
1. Read & display grayscale.
2. Identify noise type.
3. Remove noise with filtering.
4. Display result.

In [ ]:
# Read and display
img5 = cv2.imread('/mnt/data/pcb.png', cv2.IMREAD_GRAYSCALE)
show(img5, cmap='gray', title='pcb.tiff Original')

# Histogram
plt.figure()
plt.hist(img5.ravel(), bins=256)
plt.title('Histogram')
plt.show()
print("Noise: salt-and-pepper")

# Filtering
den = cv2.medianBlur(img5, 5)
den = cv2.GaussianBlur(den, (5,5), 0)
show(den, cmap='gray', title='Denoised PCB')


## Problem 6: Enhancement of pollen.tiff
**Tasks:**
1. Read & display grayscale.
2. Comment on problem.
3. Two approaches (homomorphic, CLAHE).
4. Display & justify.

In [ ]:
# Read & display
img6 = cv2.imread('/mnt/data/pollen.png', cv2.IMREAD_GRAYSCALE)
show(img6, cmap='gray', title='pollen.tiff Original')

print("Issue: uneven illumination & low contrast")

# Approach A: Homomorphic
log_img = np.log1p(np.float32(img6))
fft = np.fft.fft2(log_img)
fft_shift = np.fft.fftshift(fft)
rows, cols = img6.shape
crow, ccol = rows//2, cols//2
sigma = 30
Hh = np.zeros((rows,cols), np.float32)
for u in range(rows):
    for v in range(cols):
        D = np.hypot(u-crow, v-ccol)
        Hh[u,v] = 1 - np.exp(-(D**2)/(2*sigma**2))
filtered = Hh * fft_shift
ifft_shift = np.fft.ifftshift(filtered)
img_hom = np.fft.ifft2(ifft_shift)
img_hom = np.expm1(np.real(img_hom))
img_hom = cv2.normalize(img_hom, None, 0,255, cv2.NORM_MINMAX).astype(np.uint8)
show(img_hom, cmap='gray', title='Homomorphic')

# Approach B: CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
img_clahe = clahe.apply(img6)
show(img_clahe, cmap='gray', title='CLAHE')
print("Homomorphic fixes shading; CLAHE improves local contrast")
